In [23]:
!pip3 install -U meshio
!pip3 install -U numpy
!pip3 install -U gmsh
!pip3 install -U pygmsh

Requirement already up-to-date: meshio in /home/lesh/.local/lib/python3.8/site-packages (5.2.2)
Requirement already up-to-date: numpy in /home/lesh/.local/lib/python3.8/site-packages (1.21.5)
Requirement already up-to-date: gmsh in /home/lesh/.local/lib/python3.8/site-packages (4.9.1)
Requirement already up-to-date: pygmsh in /home/lesh/.local/lib/python3.8/site-packages (7.1.14)


In [24]:
import gmsh
import meshio
import numpy
import pygmsh

In [25]:
resolution = 0.01
LT = 10
LB = 10
W = 5

point0 = (0, 0, 0)
point1 = (LB, 0, 0)
point2 = (LT, W, 0)
point3 = (0, W, 0)
c = (2.5, 2.5, 0)
r = 0.5

In [26]:
geometry = pygmsh.geo.Geometry()
model = geometry.__enter__()

In [27]:
points = [model.add_point(point0, mesh_size=resolution),
          model.add_point(point1, mesh_size=resolution),
          model.add_point(point2, mesh_size=resolution),
          model.add_point(point3, mesh_size=resolution),
          ]

In [28]:
channel_lines = [model.add_line(points[i], points[i+1])
  for i in range(-1, len(points)-1)]
channel_loop = model.add_curve_loop(channel_lines)
circle = model.add_circle(c, r, mesh_size=resolution)
plane_surface = model.add_plane_surface(channel_loop, holes=[circle.curve_loop])
model.synchronize()

In [29]:
volume_marker = 6
model.add_physical([plane_surface], "Volume")
model.add_physical([channel_lines[0]], "Left")
model.add_physical([channel_lines[2]], "Right")
model.add_physical([channel_lines[1], channel_lines[3]], "Insulated")
model.add_physical(circle.curve_loop.curves, "void")

In [30]:
geometry.generate_mesh(dim=2)
gmsh.write("mesh.msh")
gmsh.clear()
geometry.__exit__()

In [31]:
mesh_from_file = meshio.read("mesh.msh")

In [32]:
def create_mesh(mesh, cell_type, prune_z=False):
    cells = mesh.get_cells_type(cell_type)
    cell_data = mesh.get_cell_data("gmsh:physical", cell_type)
    points = mesh.points[:,:2] if prune_z else mesh.points
    out_mesh = meshio.Mesh(points=points, cells={cell_type: cells}, cell_data={"name_to_read":[cell_data]})
    return out_mesh

In [33]:
line_mesh = create_mesh(mesh_from_file, "line", prune_z=True)
meshio.write("facet_mesh.xdmf", line_mesh)

triangle_mesh = create_mesh(mesh_from_file, "triangle", prune_z=True)
meshio.write("mesh.xdmf", triangle_mesh)
